In [1]:
correction_list = {
    "Kim K": "Kim K.",
    "Al Ghareeb M.": "Al-Ghareeb M.",
    "Al Khulaifi N.G.": "Al-Khulaifi N.G.",
    "Alawadhi O.": "Al-Awadhi O.",
    "Bahrouzyan O.": "Al-Awadhi O.",
    "Haji A.": "Al-Haji A.",
    "Hajji A.": "Al-Haji A.",
    "El Aarej M.": "El-Aarej M.",
    "El Amrani R.": "El-Amrani R.",
    "El Aynaoui Y.": "El-Aynaoui Y.",
    "Granollers Pujol G.": "Granollers-Pujol G.",
    "Granollers G.": "Granollers-Pujol G.",
    "Granollers M.": "Granollers-Pujol M.",
    "Al Mutawa J.": "Ali Mutawa J.M.",
    "Al-Mutawa J.": "Ali Mutawa J.M.",
    "Bogomolov Jr. A.": "Bogomolov Jr.A.",
    "Bogomolov A.": "Bogomolov Jr.A.",
    "Carreno Busta": "Carreno-Busta",
    "Chela J.": "Chela J.I.",
    "Dutra Silva R.": "Dutra da Silva R.",
    "Dutra Da Silva R.": "Dutra da Silva R.",
    "Ferrero J.": "Ferrero J.C.",
    "Gambill J. M.": "Gambill J.M.",
    "Guzman J.": "Guzman J.P.",
    "Haider-Mauer A.": "Haider-Maurer A.",
    "Zayid M. S.": "Zayid M.S.",
    "De Bakker T.": "de Bakker T.",
    "De Chaunac S.": "de Chaunac S.",
    "De Greef A.": "de Greef A.",
    "De Heart R.": "de Heart R.",
    "De Loore J.": "de Loore J.",
    "De Minaur A.": "de Minaur A.",
    "De Schepper K.": "de Schepper K.",
    "De Voest R.": "de Voest R.",
    "Del Bonis F.": "del Bonis F.",
    "Del Potro J.": "del Potro J.M.",
    "Del Potro J. M.": "del Potro J.M.",
    "Del Potro J.M.": "del Potro J.M.",
    "Van D. Merwe I.": "van D. Merwe I.",
    "Van Der Merwe I.": "van Der Merwe I.",
    "Van Lottum J.": "van Lottum J.",
    "Van Rijthoven T.": "van Rijthoven T.",
    "Van der Dium A.": "van der Dium A.",
    "Van der Merwe I.": "van der Merwe I.",
    "Munoz De La Nava D.": "Munoz de la Nava D.",
    "Munoz de La Nava D.": "Munoz de la Nava D.",
    "Munoz-De La Nava D.": "Munoz de la Nava D.",
    "Di Mauro A.": "di Mauro A.",
    "Di Pasquale A.": "di Pasquale A.",
}

drop_list = ["W1", "L1", "W2", "L2", "W3", "L3", "W4", "L4", "W5", "L5", "Wsets", "Lsets", "CBW", "CBL", "GBW", "GBL", "IWW", "IWL", "SBW", "SBL", "B&WW", "B&WL", "EXW", "EXL", "UBW", "UBL", "LBW", "LBL", "SJW", "SJL", "MaxW", "MaxL", "AvgW", "AvgL", "WRank", "LRank"]

Let's download, save, cleanup, and agregate the data into a dataframe and and save it to a csv file. We also caculate the elo rate of each player right before the game as well as the probability for the winner to win the game based on the elo rates of the two players.

In [ ]:
import pandas as pd
import numpy as np
import requests
import io
import datetime

# Download the data
url_base = "http://tennis-data.co.uk/"

sess = requests.session()
df_list = []
for year in range(2000, 2013):
    url = f"{url_base}/{year}/{year}.xls"
    resp = sess.get(url)
    with open(f"archive/tournoments_{year}.xls", mode="wb") as file:
        file.write(resp.content)
    df_list.append(pd.read_excel(io.BytesIO(resp.content), sheet_name=f"{year}", engine="calamine"))

for year in range(2013, datetime.datetime.now().year+1):
    url = f"{url_base}/{year}/{year}.xlsx"
    resp = sess.get(url)
    with open(f"archive/tournoments_{year}.xlsx", mode="wb") as file:
        file.write(resp.content)
    df_list.append(pd.read_excel(io.BytesIO(resp.content), sheet_name=f"{year}", engine="calamine"))
sess.close()

# Put the downloaded data together
df_atp = pd.concat(df_list, axis=0).drop(labels=drop_list, axis=1) 

# Cleanup the data
df_atp["Winner"] = df_atp["Winner"].str.strip()
df_atp["Loser"] = df_atp["Loser"].str.strip()

df_atp["Winner"] = df_atp["Winner"].replace(correction_list)
df_atp["Loser"] = df_atp["Loser"].replace(correction_list)

df_atp["Best of"] = pd.to_numeric(df_atp["Best of"], errors="coerce", downcast="integer")
df_atp["WPts"] = pd.to_numeric(df_atp["WPts"], errors="coerce", downcast="integer")
df_atp["LPts"] = pd.to_numeric(df_atp["LPts"], errors="coerce", downcast="integer")

df_atp["Date"] = pd.to_datetime(df_atp["Date"])

# Calculate the elo rate and number of played matches for each player at the begining of the match.
# Also calculate the probability for the winner of the game to win the game based on the elo rates
elo_start = 1500.0
k_factor = 32.0

df_atp[["match_count_winner", "match_count_loser"]] = 0
df_atp[["elo_winner", "elo_loser"]] = elo_start
df_atp["proba_elo"] = .5

df_atp = df_atp.sort_values(by=["ATP", "Date"]).reset_index(drop=True).copy()

elo_rates = pd.DataFrame(data={"Player": pd.concat([df_atp["Winner"], df_atp["Loser"]], axis=0).unique(), "pts": 0, "match_count":  0, "elo": elo_start})
for index, _ in df_atp.iterrows():
    # Set atp points
    wpts = df_atp.loc[index, "WPts"]
    if np.isnan(wpts):
        df_atp.loc[index, "WPts"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "pts"].values[0]
    else:
        elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "pts"] = wpts
    
    lpts = df_atp.loc[index, "LPts"]
    if np.isnan(lpts):
        df_atp.loc[index, "LPts"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "pts"].values[0]
    else:
        elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "pts"] = lpts

    # Set the current elo rate/match count for the winner and loser of the match
    df_atp.loc[index, "elo_winner"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "elo"].values[0]
    df_atp.loc[index, "elo_loser"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "elo"].values[0]
    df_atp.loc[index, "match_count_winner"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "match_count"].values[0]
    df_atp.loc[index, "match_count_loser"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "match_count"].values[0]

    # Set the probability of the outcome based on the elo rates
    df_atp.loc[index, "proba_elo"] = 1.0/(1.0 + 10.0**(.0025*(df_atp.loc[index, "elo_loser"] - df_atp.loc[index, "elo_winner"])))

    # Update the elo scores/match counts based on the outcome of the match
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "elo"] += k_factor*(1.0 - df_atp.loc[index, "proba_elo"])
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "elo"] -= k_factor*(1.0 - df_atp.loc[index, "proba_elo"])
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], "match_count"] += 1
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], "match_count"] += 1

# Save the result in respective files
df_atp.to_csv("archive/atp_data.csv")
elo_rates.to_csv("archive/elo_rates.csv")

Here, I add more features to the dataset, namely, match count and elo rates with respect to the specific court and surface type in which the game is played.

In [ ]:
import pandas as pd
import numpy as np

# Load data from the disk
df_atp = pd.read_csv("archive/atp_data.csv", index_col=0)
elo_rates = pd.read_csv("archive/elo_rates.csv", index_col=0)

court_surface_type = df_atp[["Court", "Surface"]].drop_duplicates().sort_values(by="Court").to_numpy().tolist()

try:
    elo_start
except NameError:
    elo_start = 1500.0
    k_factor = 32.0

# Initialize new columns in datasets
for pair in court_surface_type:
    col_name = f"{pair[0].lower()}_{pair[1].lower()}"
    df_atp[[f"match_count_{col_name}_winner", f"match_count_{col_name}_loser"]] = 0
    df_atp[[f"elo_{col_name}_winner", f"elo_{col_name}_loser"]] = elo_start
    df_atp[f"proba_elo_{col_name}"] = .5
    

    elo_rates[f"match_count_{col_name}"] = 0
    elo_rates[f"elo_{col_name}"] = elo_start

# Add the field specific elo rates and played match count of the players
for index, _ in df_atp.iterrows():
    field_type = f"{df_atp.loc[index, 'Court'].lower()}_{df_atp.loc[index, 'Surface'].lower()}"
    for pair in court_surface_type:
        col_name = f"{pair[0].lower()}_{pair[1].lower()}"

        # Set the current match count for the winner and loser of the match (court/surface specific)
        df_atp.loc[index, f"match_count_{col_name}_winner"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], f"match_count_{col_name}"].values[0]
        df_atp.loc[index, f"match_count_{col_name}_loser"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], f"match_count_{col_name}"].values[0]

        # Set the current elo rate of the winner and loser of the match (court/surface specific)
        df_atp.loc[index, f"elo_{col_name}_winner"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], f"elo_{col_name}"].values[0]
        df_atp.loc[index, f"elo_{col_name}_loser"] = elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], f"elo_{col_name}"].values[0]
        
        # Set the probability of the outcome based on the elo rates (court/surface specific)
        df_atp.loc[index, f"proba_elo_{col_name}"] = 1.0/(1.0 + 10.0**(.0025*(df_atp.loc[index, f"elo_{col_name}_loser"] - df_atp.loc[index, f"elo_{col_name}_winner"])))
    
    # Update match counts based on the outcome of the match (court/surface specific)
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], f"match_count_{field_type}"] += 1
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], f"match_count_{field_type}"] += 1

    # Update elo rates based on the outcome of the match (court/surface specific)
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Winner"], f"elo_{field_type}"] += k_factor*(1.0 - df_atp.loc[index, f"proba_elo_{field_type}"])
    elo_rates.loc[elo_rates["Player"]==df_atp.loc[index, "Loser"], f"elo_{field_type}"] -= k_factor*(1.0 - df_atp.loc[index, f"proba_elo_{field_type}"])

# Save the new dataset (enriched dataset)
df_atp.to_csv("archive/atp_data_enriched.csv")
elo_rates.to_csv("archive/elo_rates_enriched.csv")

# Show its info
df_atp.info()

In [ ]:
elo_rates.info()